In [1]:
import sys
!{sys.executable} -m pip install scikit-learn

In [2]:
import sys
!{sys.executable} -m pip install scikit-optimize

In [3]:
import sys
!{sys.executable} -m pip install scikeras

In [41]:
import numpy as np

In [42]:
X_train = np.load('PCA_Robust/X_train_pca.npy')
X_test = np.load('PCA_Robust/X_test_pca.npy')
y_train = np.load('PCA_Robust/y_train.npy')
y_test = np.load('PCA_Robust/y_test.npy')

In [43]:
from sklearn.model_selection import RepeatedStratifiedKFold
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=10, random_state=41)

In [73]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import activations
from tensorflow.keras import layers
#from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from scikeras.wrappers import KerasClassifier

In [74]:
def create_model(hidden_layer_dim, meta):
#    n_features_in_ = meta["n_features_in_"]
#    X_train_shape_ = meta["X_train_shape_"]
#    n_classes_ = meta["n_classes_"]

    model = Sequential()
#    model.add(Dense(n_features_in_, input_shape=X_train_shape_[1:]))
    model.add(layers.Activation("relu"))
    model.add(layers.Dense(hidden_layer_dim))
    model.add(layers.Activation("relu"))
#    model.add(Dense(n_classes_))
    model.add(layers.Activation("softmax"))
    return model

In [75]:
# Create an instance of the KerasClassifier using the defined model function
keras_classifier = KerasClassifier(create_model, verbose=0,loss="sparse_categorical_crossentropy",hidden_layer_dim=100)

In [76]:
keras_classifier.get_params().keys()

dict_keys(['model', 'build_fn', 'warm_start', 'random_state', 'optimizer', 'loss', 'metrics', 'batch_size', 'validation_batch_size', 'verbose', 'callbacks', 'validation_split', 'shuffle', 'run_eagerly', 'epochs', 'hidden_layer_dim', 'class_weight'])

In [77]:
params = {
    'hidden_layer_dim': [250, 300, 350, 400],
    'optimizer': ['adamax', 'sgd', 'adam'],
    'loss': ['sparse_categorical_crossentropy'],
    'optimizer__learning_rate': np.logspace(-5, -2, base=10)
}


In [80]:
# Create an instance of BayesianSearchCV
random_cv = RandomizedSearchCV(keras_classifier, params, n_iter=1000, cv=cv, n_jobs=-1)

In [ ]:
# Perform hyperparameter tuning using Bayesian optimization
random_cv.fit(X_train, np.ravel(y_train))

/opt/saturncloud/envs/saturn/lib/python3.9/site-packages/sklearn/model_selection/_search.py:292: UserWarning: The total space of parameters 600 is smaller than n_iter=1000. Running 600 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


In [ ]:
# Print the best score and hyperparameters
print("Train score:", bayes_cv.best_score_)
print("Test score:", bayes_cv.score(X_test, y_test))
print("Best hyperparameters:", bayes_cv.best_params_)

In [ ]:
keras.l